In [2]:
#Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("data/GoalData.csv")
print(df.head)

<bound method NDFrame.head of      Wk  Day        Date           Time           Home  HomexG Score  AwayxG  \
0     1  Fri  2023-08-11  20:00 (14:00)        Burnley     0.3   0–3     1.9   
1     1  Sat  2023-08-12  12:30 (06:30)        Arsenal     0.8   2–1     1.2   
2     1  Sat  2023-08-12  15:00 (09:00)        Everton     2.7   0–1     1.5   
3     1  Sat  2023-08-12  15:00 (09:00)  Sheffield Utd     0.5   0–1     1.9   
4     1  Sat  2023-08-12  15:00 (09:00)       Brighton     4.0   4–1     1.5   
..   ..  ...         ...            ...            ...     ...   ...     ...   
273  28  Sun  2024-03-10  13:00 (08:00)    Aston Villa     0.8   0–4     1.5   
274  28  Sun  2024-03-10  14:00 (09:00)       West Ham     2.1   2–2     0.7   
275  28  Sun  2024-03-10  14:00 (09:00)       Brighton     0.4   1–0     0.7   
276  28  Sun  2024-03-10  15:45 (10:45)      Liverpool     2.7   1–1     1.6   
277  28  Mon  2024-03-11  20:00 (15:00)        Chelsea     1.6   3–2     0.7   

         

Data Pre-Processing

In [4]:
def gather_home_goals(value):
    if value != None:
        return int(value[0])
    pass

def gather_away_goals(value):
    if value != None:
        return int(value[2])
    
def gather_winner(value):
    if(value!=None):
        if(int(value[0]) > int(value[2])):
            return 'H'
        elif(int(value[2])>int(value[0])):
            return "A"
        return 'T'
columns_to_include = ['HomexG', 'AwayxG', 'Score']
df = df[columns_to_include]

df['HomeG'] = df['Score'].apply(lambda x: gather_home_goals(x))
df['AwayG'] = df['Score'].apply(lambda x: gather_away_goals(x))
df['DiffxG'] = df['HomexG'] - df['AwayxG']
df['Result'] = df['Score'].apply(lambda x: gather_winner(x))

print(df.head)

<bound method NDFrame.head of      HomexG  AwayxG Score  HomeG  AwayG  DiffxG Result
0       0.3     1.9   0–3      0      3    -1.6      A
1       0.8     1.2   2–1      2      1    -0.4      H
2       2.7     1.5   0–1      0      1     1.2      A
3       0.5     1.9   0–1      0      1    -1.4      A
4       4.0     1.5   4–1      4      1     2.5      H
..      ...     ...   ...    ...    ...     ...    ...
273     0.8     1.5   0–4      0      4    -0.7      A
274     2.1     0.7   2–2      2      2     1.4      T
275     0.4     0.7   1–0      1      0    -0.3      H
276     2.7     1.6   1–1      1      1     1.1      T
277     1.6     0.7   3–2      3      2     0.9      H

[278 rows x 7 columns]>


KNN Modeling

In [6]:
X = df[['HomeG', 'AwayG', 'DiffxG']]
y = df['Result']
print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")

     HomeG  AwayG  DiffxG
0        0      3    -1.6
1        2      1    -0.4
2        0      1     1.2
3        0      1    -1.4
4        4      1     2.5
..     ...    ...     ...
273      0      4    -0.7
274      2      2     1.4
275      1      0    -0.3
276      1      1     1.1
277      3      2     0.9

[278 rows x 3 columns]
0      A
1      H
2      A
3      A
4      H
      ..
273    A
274    T
275    H
276    T
277    H
Name: Result, Length: 278, dtype: object
Model accuracy: 0.96


Test Predictions

In [7]:
test_data_df = pd.DataFrame({
    'HomeG': [2],
    'AwayG': [0],
    'DiffxG': [1.5 - 0.9]
})

predicted_result = knn.predict(test_data_df)
print(f"Predicted Result: {predicted_result[0]}")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- AwayxG
- HomexG
Feature names seen at fit time, yet now missing:
- AwayG
- HomeG


Conclusion